In [6]:
import sys

In [8]:
sys.path.insert(0, "../")

In [17]:
import os
import pandas as pd
from utils.data_mapping import id_from_filename
import cv2
import yaml

In [10]:
data = "/datasets/processed_256_test"

In [11]:
img_fp = list(os.listdir(os.path.join(data, "img")))
ids = [id_from_filename(fn) for fn in img_fp]
img_fp = [os.path.join(data, "img", fn) for fn in img_fp]

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/processed_256_test/img'

In [ ]:
from dataloaders.transformations import create_transform
best_model_path = "experiments/"
best_model_dir = os.path.dirname(best_model_path)
with open(os.path.join(best_model_dir, 'config.yaml'), 'r') as f:
    config = yaml.load(f)
transform_config = config["transformations"]
transform = create_transform(transform_config["image"])

In [18]:
from models.classifier_unet import UNet
model = UNet()
model.load_state_dict(torch.load(best_model_path, map_location="cpu"))
model.eval()

ImportError: No module named 'pretrainedmodels'

In [19]:
import torch

In [16]:
full = 1024*1024
submissions = []
for img_id, single_img_fp in zip(ids, img_fp):
    img = cv2.imread(single_img_fp, cv2.IMREAD_GRAYSCALE)
    horizontal_img = cv2.flip( img, 0 )
    vertical_img = cv2.flip( img, 1 )
    both_img = cv2.flip( img, -1 )
    imgs = [img, horizontal_img, vertical_img, both_img]
    imgs = [transform(img) for img in imgs]
    imgs = torch.Tensor(imgs).float()
    with torch.no_grad():
        _, out = model.forward(model)
        out = out.argmax(1).float().mean()
    out_from_nn = out # zero or one

    pred = "{} 1".format(full)
    if out_from_nn <= 0.5: # zero
        pred = "-1"
    submissions.append([img_id, pred])

submission_df = pd.DataFrame(submissions, columns=['ImageId', 'EncodedPixels'])

NameError: name 'ids' is not defined

In [15]:
out_fn = "bodya_classifier.csv"
submission_df.to_csv(out_fn, index=False)

NameError: name 'submission_df' is not defined